# Project2 : Analyse des Maladies de l’Arachide à l’Aide d’algorithmes d’intelligence artificielle.

* Abdou Aziz Kane
* Maguette Leye
* Aboubacar Sadikh Sow
* Abdoukhadre Sylla

### Importation des bibliothèques nécessaires
* Cette section importe les modules nécessaires pour la manipulation des données, la création de modèles et l'entraînement.

In [8]:
import os
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image

### Définition du chemin des données
* Définit le chemin où sont stockées les images pour l'entraînement et la validation.

In [9]:
data_dir = r"C:\Users\abous\Desktop\DATASET\train"

### Définition des transformations des données
* Applique des transformations pour augmenter, redimensionner et normaliser les données.

In [10]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### Chargement et division des données
* Charge les images depuis le dossier et les divise en ensembles d'entraînement, validation et test.

In [11]:
dataset = ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

### Définition du modèle CNN amélioré
* Déclare une classe CNN améliorée pour la classification des maladies des feuilles d'arachide.

In [12]:
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes):  # Correction de __init__
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(512 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

### Initialisation du modèle et des paramètres d'entraînement
* Crée une instance du modèle et définit le périphérique d'exécution (GPU ou CPU).

In [13]:
num_classes = len(dataset.classes)
model = ImprovedCNN(num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)


### Définition de la fonction d'entraînement
* Entraîne le modèle, affiche la perte et sauvegarde le meilleur modèle en fonction de la précision.

In [14]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs):
    best_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")
        
        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f"Validation Accuracy: {accuracy:.2f}%")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), "best_model.pth")
        
        scheduler.step()
    
    return best_accuracy

### Exécution de l'entraînement du modèle
* Lance l'entraînement du modèle et affiche la meilleure précision atteinte.

In [15]:
best_acc = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=100)
print(f"Meilleure précision: {best_acc:.2f}%")

Epoch 1, Loss: 1.7797
Validation Accuracy: 39.44%
Epoch 2, Loss: 1.4463
Validation Accuracy: 34.64%
Epoch 3, Loss: 1.3558
Validation Accuracy: 42.86%
Epoch 4, Loss: 1.2888
Validation Accuracy: 50.70%
Epoch 5, Loss: 1.2263
Validation Accuracy: 53.73%
Epoch 6, Loss: 1.1198
Validation Accuracy: 63.59%
Epoch 7, Loss: 1.0383
Validation Accuracy: 58.41%
Epoch 8, Loss: 0.9708
Validation Accuracy: 64.48%
Epoch 9, Loss: 0.8689
Validation Accuracy: 71.93%
Epoch 10, Loss: 0.8098
Validation Accuracy: 70.67%
Epoch 11, Loss: 0.7385
Validation Accuracy: 65.99%
Epoch 12, Loss: 0.7030
Validation Accuracy: 74.34%
Epoch 13, Loss: 0.6518
Validation Accuracy: 80.15%
Epoch 14, Loss: 0.5874
Validation Accuracy: 82.68%
Epoch 15, Loss: 0.5761
Validation Accuracy: 81.54%
Epoch 16, Loss: 0.5550
Validation Accuracy: 83.06%
Epoch 17, Loss: 0.5249
Validation Accuracy: 82.43%
Epoch 18, Loss: 0.5159
Validation Accuracy: 82.93%
Epoch 19, Loss: 0.4978
Validation Accuracy: 86.09%
Epoch 20, Loss: 0.4932
Validation Accura

#####
### Cette partie charge un modèle de réseau de neurones convolutif (CNN) pré-entraîné pour classifier les maladies des feuilles d'arachide à partir d'images
#####

### Importation des bibliothèques nécessaires
* Cette section importe les modules requis pour la manipulation des images et l'utilisation du modèle.

In [16]:
import os
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F


### Chargement du modèle entraîné
* Définition du chemin du modèle sauvegardé et du nombre de classes

In [17]:
model_path = r"C:\Users\abous\Desktop\best_model.pth"
num_classes = 6  # Modifier selon le nombre de classes

### Définition du modèle CNN
* Création d'un modèle CNN simple pour la classification des maladies des feuilles d'arachide

In [18]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):  # Correction de l'initialisation
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self._to_linear = None
        self.convs(torch.zeros(1, 3, 128, 128))  # Dummy pass pour calculer la taille
        self.fc1 = nn.Linear(self._to_linear, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def convs(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        if self._to_linear is None:
            self._to_linear = x.numel()
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Chargement du modèle pré-entraîné
* Chargement du modèle et mise en mode évaluation

In [21]:
model = ImprovedCNN(num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'), weights_only=True))
model.eval()


ImprovedCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=32768, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=6, bias=True)


### Définition des transformations de l'image
* Transformation pour adapter les images au format du modèle

In [22]:
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### Fonction de prédiction
* Prend une image en entrée, applique la transformation et retourne la classe prédite avec la confiance

In [23]:
def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    
    with torch.no_grad():
        output = model(image)
        probabilities = torch.softmax(output, dim=1)  # Convertir en probabilités
        confidence, predicted = torch.max(probabilities, 1)  # Probabilité max et index associé
    
    class_names = ["early_leaf_spot_1","early_rust_1","healthy_leaf_1","late_leaf_spot_1","nutrition_deficiency_1","rust_1"]  # Modifier selon les classes
    return class_names[predicted.item()], confidence.item()


### Exemple d'utilisation
* Charger une image et obtenir la prédiction

In [27]:
image_path = r'C:\Users\abous\Desktop\test_22.jpg'  # Modifier avec le chemin de votre image
disease, confidence = predict(image_path)
print(f"La plante est classifiée comme : {disease} (Confiance : {confidence:.2%})")

La plante est classifiée comme : healthy_leaf_1 (Confiance : 92.92%)


#####
### Cette partie implémente une interface graphique avec customtkinter pour détecter les maladies des feuilles d’arachide à partir d’images puis on crée le transforme en fichier executable (.exe) avec le modele inclus
#####

#####
### Lien de telechargement de l'application :
https://drive.google.com/file/d/1Famvx92P9CZFCI4opPrR0B72mYP2VrNY/view?usp=sharing
#####

#####
### Lien de telechargement du code source de l'application et le modele  :
https://drive.google.com/file/d/1OkJ5qlP7QveTDmm9fWL9GgMqf0WcNOdU/view?usp=sharing
#####

### Vous Pouvez utiliser le dossier "test" pour prendre des images et tester l'appliquation.